In [4]:
import logging
import pandas as pd

from matplotlib import pyplot as plt
from conf import download_conf
from utils.paths import tp

tickers = download_conf.tickers
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [2]:
dfs = []
for i, ticker in enumerate(tickers):
    if i % 10 == 0:
        logger.info(f"Loaded {i} / {len(tickers)}.")
    path = tp(ticker, granularity="1d")
    df = pd.read_csv(path)
    df["ticker"] = ticker
    dfs.append(df)
df_raw = pd.concat(dfs)
df_raw

INFO:__main__:Loaded 0 / 835.
INFO:__main__:Loaded 10 / 835.
INFO:__main__:Loaded 20 / 835.
INFO:__main__:Loaded 30 / 835.
INFO:__main__:Loaded 40 / 835.
INFO:__main__:Loaded 50 / 835.
INFO:__main__:Loaded 60 / 835.
INFO:__main__:Loaded 70 / 835.
INFO:__main__:Loaded 80 / 835.
INFO:__main__:Loaded 90 / 835.
INFO:__main__:Loaded 100 / 835.
INFO:__main__:Loaded 110 / 835.
INFO:__main__:Loaded 120 / 835.
INFO:__main__:Loaded 130 / 835.
INFO:__main__:Loaded 140 / 835.
INFO:__main__:Loaded 150 / 835.
INFO:__main__:Loaded 160 / 835.
INFO:__main__:Loaded 170 / 835.
INFO:__main__:Loaded 180 / 835.
INFO:__main__:Loaded 190 / 835.
INFO:__main__:Loaded 200 / 835.
INFO:__main__:Loaded 210 / 835.
INFO:__main__:Loaded 220 / 835.
INFO:__main__:Loaded 230 / 835.
INFO:__main__:Loaded 240 / 835.
INFO:__main__:Loaded 250 / 835.
INFO:__main__:Loaded 260 / 835.
INFO:__main__:Loaded 270 / 835.
INFO:__main__:Loaded 280 / 835.
INFO:__main__:Loaded 290 / 835.
INFO:__main__:Loaded 300 / 835.
INFO:__main__:Loade

,date,open,high,low,close,adj_close,volume,dividends,stock_splits,ticker
0,2019-04-17 00:00:00,11.13,12.3900,10.74,11.0900,11.0900,1286600.0,0.0,0.0,BWAY
1,2019-04-18 00:00:00,11.03,11.4500,10.80,11.2500,11.2500,165800.0,0.0,0.0,BWAY
2,2019-04-22 00:00:00,11.40,11.5000,11.30,11.5000,11.5000,50200.0,0.0,0.0,BWAY
3,2019-04-23 00:00:00,11.55,11.5500,11.04,11.3100,11.3100,26300.0,0.0,0.0,BWAY
4,2019-04-24 00:00:00,11.22,11.2800,11.00,11.2500,11.2500,20900.0,0.0,0.0,BWAY
...,...,...,...,...,...,...,...,...,...,...
1009,2023-02-13 00:00:00,1.28,1.3199,1.26,1.3193,1.3193,274673.0,0.0,0.0,CTRM
1010,2023-02-14 00:00:00,1.28,1.3200,1.28,1.3200,1.3200,137757.0,0.0,0.0,CTRM
1011,2023-02-15 00:00:00,1.32,1.3300,1.31,1.3250,1.3250,62003.0,0.0,0.0,CTRM
1012,2023-02-16 00:00:00,1.31,1.3202,1.29,1.2950,1.2950,216836.0,0.0,0.0,CTRM


In [31]:
import datetime
import numpy as np
start_date = datetime.datetime(2000, 1, 1, 0, 0, 0)
df = df_raw
df.date = df_raw.date.astype(np.datetime64)
df_tickers = df_raw.groupby("ticker")["date"].agg([min, max]).reset_index()
df_tickers = df_tickers[(df_tickers["min"] <= start_date) & (df_tickers["max"] >= datetime.datetime.now() - datetime.timedelta(days=4))][["ticker"]]
tickers = df_tickers.ticker.values
logger.info(f"{tickers.shape[0]} were there since 2000.")
df_tickers = df_raw.groupby("ticker")["volume"].agg("mean").reset_index()
df_tickers = df_tickers[df_tickers.volume < 50000]
tickers = np.array(list(set(list(tickers)) - set(list(df_tickers.ticker.values))))
logger.info(f"{tickers.shape[0]} were there since 2000 and had more than 50K transactions per day in average.")
df_tickers = df_raw.groupby('ticker')["open"].apply(lambda x: any(x.isna())).reset_index()
df_tickers = df_tickers[df_tickers.open]
tickers = np.array(list(set(list(tickers)) - set(list(df_tickers.ticker.values))))
logger.info(f"{tickers.shape[0]} were there since 2000 and had more than 50K transactions per day in average and hadn't any none.")
df_tickers = pd.DataFrame(tickers, columns = ["ticker"])
df = df.join(df_tickers.set_index("ticker"), how = "inner", on = "ticker")
df_norm = df_raw.groupby('ticker')["open"].apply(lambda x: ((x.values[1:] + 1) / (x.values[:-1] + 1)).max() < 5).reset_index()
tickers = df_norm[df_norm.open].ticker.values
df_tickers = pd.DataFrame(tickers, columns = ["ticker"])
df = df.join(df_tickers.set_index("ticker"), how = "inner", on = "ticker")
logger.info(f"{df.groupby('ticker').size().reset_index().shape[0]} were there since 2000 and had more than 50K transactions per day in average and hadn't any none and didn't do more than x20.")
df = df[df.date >= start_date]
df

INFO:__main__:205 were there since 2000.
INFO:__main__:150 were there since 2000 and had more than 50K transactions per day in average.
INFO:__main__:149 were there since 2000 and had more than 50K transactions per day in average and hadn't any none.
INFO:__main__:130 were there since 2000 and had more than 50K transactions per day in average and hadn't any none and didn't do more than x20.


,date,open,high,low,close,adj_close,volume,dividends,stock_splits,ticker
524,2000-01-03,24.5000,24.7500,21.5000,22.0000,22.0000,12950.0,0.0,0.0,ASRT
525,2000-01-04,22.5000,23.0000,21.5000,22.5000,22.5000,6275.0,0.0,0.0,ASRT
526,2000-01-05,22.2500,22.2500,22.0000,22.0000,22.0000,3575.0,0.0,0.0,ASRT
527,2000-01-06,21.5000,22.0000,20.5000,21.5000,21.5000,2675.0,0.0,0.0,ASRT
528,2000-01-07,22.0000,22.0000,20.2500,21.5000,21.5000,7725.0,0.0,0.0,ASRT
...,...,...,...,...,...,...,...,...,...,...
9686,2023-02-13,2.8800,2.9200,2.8800,2.9100,2.9100,14709.0,0.0,0.0,BMRA
9687,2023-02-14,2.8550,2.8800,2.8550,2.8800,2.8800,1278.0,0.0,0.0,BMRA
9688,2023-02-15,2.9253,2.9253,2.9000,2.9000,2.9000,1064.0,0.0,0.0,BMRA
9689,2023-02-16,2.9713,2.9900,2.9401,2.9401,2.9401,7670.0,0.0,0.0,BMRA


In [32]:
train_date = "2022-01-01"
df_train = df[df.date <= train_date]
df_test = df[df.date > train_date]
df_test

,date,open,high,low,close,adj_close,volume,dividends,stock_splits,ticker
6060,2022-01-03,2.1900,2.7300,2.1800,2.6500,2.6500,3146900.0,0.0,0.0,ASRT
6061,2022-01-04,2.6500,2.7200,2.4700,2.6100,2.6100,1286400.0,0.0,0.0,ASRT
6062,2022-01-05,2.6000,2.6200,2.3600,2.4500,2.4500,943700.0,0.0,0.0,ASRT
6063,2022-01-06,2.4600,2.5800,2.3700,2.4100,2.4100,565000.0,0.0,0.0,ASRT
6064,2022-01-07,2.4400,2.8200,2.4200,2.7100,2.7100,2168900.0,0.0,0.0,ASRT
...,...,...,...,...,...,...,...,...,...,...
9686,2023-02-13,2.8800,2.9200,2.8800,2.9100,2.9100,14709.0,0.0,0.0,BMRA
9687,2023-02-14,2.8550,2.8800,2.8550,2.8800,2.8800,1278.0,0.0,0.0,BMRA
9688,2023-02-15,2.9253,2.9253,2.9000,2.9000,2.9000,1064.0,0.0,0.0,BMRA
9689,2023-02-16,2.9713,2.9900,2.9401,2.9401,2.9401,7670.0,0.0,0.0,BMRA


In [33]:
cols = ["open", "high", "low", "close", "adj_close", "volume"]
def prepare(df, volume_max=None):  
    if not volume_max:
        df = df.join(df.groupby("ticker").volume.max(), on = "ticker", rsuffix="_max")
        df.volume = df.volume / df.volume_max
    else:
        df.volume = df.volume / volume_max
    df[cols] = df[cols].astype(np.float32)
    df = df.groupby("date")[cols].apply(lambda x: pd.Series({"x": np.array(x)}))["x"].reset_index()
    return df, volume_max

df_train, volume_max = prepare(df_train, volume_max=None)
df_test, _ = prepare(df_test, volume_max=volume_max)

In [ ]:
x = np.stack(df["x"].values)
for i in range(1):
    plt.plot(x[:,48,i])

In [34]:
n_days = 5
def get_data_generator(df, n_days, batch_size=0, training=True):
    n  = 0
    n_tickers = 130
    while True:
        n += 1
        logging.info(f"Restarting data for the {n}th time.")
        tot_days = df.shape[0]
        # Real number is batch_size - 2, to be investigate why.
        i_list = list(range(n_days + batch_size, tot_days - batch_size))
        if training:
            np.random.shuffle(i_list)
        for i in range(batch_size, len(i_list) - batch_size, batch_size):
            X = []
            Y = []
            for j in range(batch_size):
                idx = i_list[i + j]
                for_zero_division = np.concatenate([np.zeros((1, n_tickers, 5)), np.ones((1, n_tickers, 1))], axis = 2)
                norm = (np.array([df.iloc[idx-1]["x"]]) + for_zero_division)
                x = np.array([np.stack(df.iloc[idx-n_days:idx]["x"].values)]) / norm
                y = (np.array([df.iloc[idx]["x"]]) / norm)[:,:,:4]
                X.append(x)
                Y.append(y)
            X = np.vstack(X)
            Y = np.vstack(Y)
            yield X, Y


batch_size = 8
train_tot_days = df_train.shape[0]
train_gen = get_data_generator(df_train, n_days, batch_size=batch_size)
train_nb_steps = len([i for i in range(batch_size, len(list(range(n_days + batch_size, train_tot_days - batch_size))) - batch_size, batch_size)])
test_tot_days = df_test.shape[0]
test_gen = get_data_generator(df_test, n_days, batch_size=batch_size, training=False)
test_nb_steps = len([i for i in range(batch_size, len(list(range(n_days + batch_size, test_tot_days - batch_size))) - batch_size, batch_size)])

In [27]:
def mse(y_pred, y_true):
    return round(np.sum((y_pred - y_true)**2) / np.prod([i for i in y_pred.shape]), 4)
test_tot_days = df_test.shape[0]
test_gen = get_data_generator(df_test, n_days, batch_size=batch_size, training=False)
test_nb_steps = len([i for i in range(batch_size, len(list(range(n_days + batch_size, test_tot_days - batch_size))) - batch_size, batch_size)])
x_test, y_test = list(zip(*[next(test_gen) for i in range(test_nb_steps)]))
x_test = np.vstack(x_test)
y_test = np.vstack(y_test)
print("Test loss baseline:" mse(x_test[:,-1,:,:4],y_test))

train_tot_days = df_train.shape[0]
train_gen = get_data_generator(df_train, n_days, batch_size=batch_size)
train_nb_steps = len([i for i in range(batch_size, len(list(range(n_days + batch_size, train_tot_days - batch_size))) - batch_size, batch_size)])
x_test, y_test = list(zip(*[next(train_gen) for i in range(train_nb_steps)]))
x_test = np.vstack(x_test)
y_test = np.vstack(y_test)
print("Test loss baseline:" mse(x_test[:,-1,:,:4],y_test))

INFO:root:Restarting data for the 1th time.


0.0014


In [37]:
import tensorflow as tf
from tensorflow.keras import layers as tfl
def get_model():
    n_tickers = 130
    input_shape = (n_days, n_tickers, len(cols),)
    output_shape = (n_tickers, 4)
    inp = tfl.Input(input_shape)
    conv = tfl.Conv2D(filters=12, kernel_size=(2,3), strides=(2, 1), padding="same", activation = None)(inp)
    conv = tfl.Conv2D(filters=10, kernel_size=(2,3), strides=(2, 1), padding="same", activation = None)(conv)
    conv = tfl.Conv2D(filters=8, kernel_size=(1,3), strides=(2, 1), padding="same", activation = None)(conv)
    conv = tfl.Conv2D(filters=6, kernel_size=(1,3), strides=(2, 1), padding="same", activation = None)(conv)
    conv = tfl.Conv2D(filters=4, kernel_size=(1,3), strides=(2, 1), padding="same", activation = None)(conv)
    #conv = tfl.Add()([conv, inp[:,-1,:,:4]])
    dense = tfl.Flatten()(conv)
#     dense = tfl.Dense(596)(dense)
#     dense = tfl.Add()([dense, (tfl.Flatten()(inp[:,-1,:,:4]))])
    output = tfl.Reshape(output_shape)(dense)
    model = tf.keras.Model(inputs=inp, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    return model

model = get_model()
h = model.fit(train_gen, validation_data=test_gen, epochs=1000, steps_per_epoch=train_nb_steps, validation_steps=test_nb_steps)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 5, 130, 6)]       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 3, 130, 12)        444       
                                                                 
 conv2d_11 (Conv2D)          (None, 2, 130, 10)        730       
                                                                 
 conv2d_12 (Conv2D)          (None, 1, 130, 8)         248       
                                                                 
 conv2d_13 (Conv2D)          (None, 1, 130, 6)         150       
                                                                 
 conv2d_14 (Conv2D)          (None, 1, 130, 4)         76        
                                                                 
 flatten_2 (Flatten)         (None, 520)               0   

INFO:root:Restarting data for the 3th time.


688/688 [==============================] - ETA: 0s - loss: 0.0184

INFO:root:Restarting data for the 32th time.


688/688 [==============================] - 9s 11ms/step - loss: 0.0184 - val_loss: 0.0045
Epoch 2/1000
684/688 [============================>.] - ETA: 0s - loss: 0.0037

...

INFO:root:Restarting data for the 57th time.


688/688 [==============================] - 8s 11ms/step - loss: 0.0016 - val_loss: 0.0011
Epoch 26/1000
684/688 [============================>.] - ETA: 0s - loss: 0.0016

INFO:root:Restarting data for the 28th time.
INFO:root:Restarting data for the 58th time.


688/688 [==============================] - 9s 14ms/step - loss: 0.0016 - val_loss: 0.0011
Epoch 27/1000
 78/688 [==>...........................] - ETA: 7s - loss: 0.0020

KeyboardInterrupt: 

In [ ]:
plt.plot(h.history["loss"][4:])

In [ ]:
def mse(y_pred, y_true):
    return round(np.sum((y_pred - y_true)**2) / np.prod([i for i in y_pred.shape]), 4)
gen = get_data_generator(df, n_days, batch_size=batch_size)
nb_steps = len([i for i in range(batch_size, len(list(range(n_days + batch_size, tot_days - batch_size))) - batch_size, batch_size)])
x_test, y_test = list(zip(*[next(gen) for i in range(nb_steps)]))
x_test = np.vstack(x_test)
y_test = np.vstack(y_test)
y_pred = model.predict(x_test)
print(mse(y_pred, y_test), mse(x_test[:,-1,:,:4],y_test))

In [ ]:
np.argmin(np.mean(((y_pred - y_test)**2), axis = 0)[:,0])

In [ ]:
gen = get_data_generator(df, n_days, batch_size=batch_size)
nb_steps = len([i for i in range(batch_size, len(list(range(n_days + batch_size, tot_days - batch_size))) - batch_size, batch_size)])
x_test, y_test = list(zip(*[next(gen) for i in range(nb_steps)]))
x_test = np.vstack(x_test)
y_test = np.vstack(y_test)
mse(x_test[:,-1,:,:4],y_test)

In [ ]:
x_test, y_test = list(zip(*[next(gen) for i in range(10)]))
x_test = np.vstack(x_test)
y_test = np.vstack(y_test)
y_pred = model.predict(x_test)
list(zip(y_pred, y_test))
model.evaluate(x_test, y_test)
print(mse(y_pred, y_test), mse(x_test[:,-1,:,:4],y_test))
plt.plot(np.concatenate([x_test[0,:,68,0], y_test[0,0,0:1]]), c="g")
plt.plot(np.concatenate([x_test[0,:,68,0], y_pred[0,0,0:1]]), c="b")

In [ ]:
ticker_id = np.random.randint(0, 205)
sample_id = 91
plt.plot(np.concatenate([x_test[sample_id,:,ticker_id,0], y_test[sample_id,ticker_id,0:1]]), c="g")
plt.plot(np.concatenate([x_test[sample_id,:,ticker_id,0], y_pred[sample_id,ticker_id,0:1]]), c="b")

In [ ]:
plt.plot([mse(y_pred[91, i], y_test[91, i]) for i in range(205)])

In [ ]:
ticker_id = np.random.randint(0, 205)
plt.plot(np.concatenate([x_test[91,:,ticker_id,0], y_test[91,ticker_id,0:1]]), c="g")
plt.plot(np.concatenate([x_test[91,:,ticker_id,0], y_pred[91,ticker_id,0:1]]), c="b")

In [ ]:
x_test.shape, y_test.shape